## Downloading and uploading data

#### Extras

* databases to download here: http://archive.eso.org/cms/eso-data/ambient-conditions.html

* source how to install mongodb: https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-os-x/

        to start mongodb in terminal: $ brew services start mongodb-community@7.0

        to stop it: $ brew services stop mongodb-community@7.0

In [ ]:
import requests
import pandas as pd
from io import StringIO
from pymongo import MongoClient

from variable_utils import  payload_paranal        

In [ ]:

def request_and_load_to_mongodb(api_url, payload, db_name):
    
    # request to endpoint
    try:
        response = requests.get(api_url, params=payload)
        print("Request status: ", response.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error in API request: {e}")

    content_lines = response.content.split(b"\n\n")

    # checking the content
    if content_lines:
        first_line = content_lines[0]
        print(first_line)
    else:
        print("No data received from the API :(")

    # set as a object file 
    csv_data = StringIO(first_line.decode('utf-8'))
    df = pd.read_csv(csv_data)
    print("Data in DF" )

    # Connection to local mongodb
    try:
        client = MongoClient("mongodb://localhost:27017/")
        db = client[db_name]
        collection = db[db_name]
        records = df.to_dict('records')
        collection.insert_many(records)
        print("Data inserted into MongoDB successfully!!")
    except Exception as e:
        print(f"Error in MongoDB operation: {e}")
    finally:
        client.close()


In [ ]:
from datetime import datetime
def get_year_range(start_year = 1998,end_year = 2023):
    "function which gets year range as a list"
    year_list = []
    for year in range(start_year, end_year + 1):
        start_date = datetime(year, 1, 1)
        end_date = datetime(year + 1, 1, 1) if year < end_year else datetime(end_year + 1, 1, 1)
        year_range_str = f"{start_date.strftime('%Y-%m-%d')}..{end_date.strftime('%Y-%m-%d')}"
        year_list.append(year_range_str)
    return year_list

year_range = get_year_range(start_year = 1998,end_year = 2024)
print(year_range[0])
print(year_range[-1])



In [ ]:
db_name = "meteo_paranal" # meteo_paranal_test, meteo_paranal
api_url_paranal = "http://archive.eso.org/wdb/wdb/asm/meteo_paranal/query"

# iterating according to the periods
for period in year_range:
    payload_paranal['start_date'] = period
    print("="*40)
    print(f"{period}\n")
    request_and_load_to_mongodb(api_url = api_url_paranal , payload = payload_paranal, db_name = db_name)
print("Done!")

In [ ]:
# checking
client = MongoClient("mongodb://localhost:27017/")
db = client[db_name] # meteo_paranal_test
collection = db[db_name]
collection.find_one({})


